# Format text from wikipedia biography

In [1]:
import pandas as pd
from _datetime import date

In [159]:
people_list = ['Glenn_Miller','Andrés_Segovia','Django_Reinhardt','Maria_Callas','Édith_Piaf','John_Lennon']
wikiId_list = ['64610','71932','9039','64966','64963','15852']
# wikiId = ['64610']

In [12]:
def readText(filename):
    fileObject = open("people/"+filename+'.txt', "r")
    data = fileObject.readlines()
    return data

def writeTextFile(filename,content):
    f = open('cleanText/'+filename+'.txt', "w")
    f.writelines(content)
    f.close()
    
def cleanText(text):
    ctext=[]
    for line in text:
        if "== See also ==" in line:
            break
        if not len(line.strip()) < 1 :
            ctext.append(line)
    return ctext

In [14]:
df = pd.DataFrame()

for person in wikiId_list:
    # read text
    text = readText(person)
    # clean empty lines
    ctext= cleanText(text)
    print(len(ctext))
    writeTextFile(person,ctext)

47
66
97
218
111
92
